In [2]:
import requests
import pandas as pd
from datetime import datetime
import schedule
import time

In [2]:
# Initialize a DataFrame to store weather data
weather_df = pd.DataFrame(columns=['timestamp', 'city', 'temperature', 'humidity', 'pressure', 'weather', 'wind_speed', 'season'])

In [3]:
# Your OpenWeatherMap API key
api_key = '429e6570bf176b57734d33f03e17ea4a'
# List of cities you want to fetch weather data for
cities = ['Delhi', 'Mumbai', 'Bangalore']

In [4]:
# Function to fetch weather data
def fetch_weather(city, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data for {city}: {response.status_code}")
        return None

# Fetch weather data for each city and print results
for city in cities:
    weather_data = fetch_weather(city, api_key)
    if weather_data:
        print(f"Weather in {city}: {weather_data['main']['temp']}°C, {weather_data['weather'][0]['description']}")

Weather in Delhi: 35.05°C, haze
Weather in Mumbai: 30.99°C, haze
Weather in Bangalore: 29.99°C, broken clouds


In [5]:
# Function to determine the current season based on Indian weather
def get_current_season():
    month = datetime.now().month
    if month in [12, 1, 2, 3]:  # Winter: December to early April
        return 'Winter'
    elif month in [4, 5, 6]:    # Summer or pre-monsoon: April to June (or July in NW India)
        return 'Summer'
    elif month in [7, 8, 9]:    # Monsoon or rainy: June to September
        return 'Monsoon'
    else:                       # Post-monsoon: October to December
        return 'Post-monsoon'


In [6]:
# Function to update weather DataFrame and save to CSV
def job():
    global weather_df
    for city in cities:
        weather_data = fetch_weather(city, api_key)
        if weather_data is not None:
            new_data = pd.DataFrame([{
                "timestamp": datetime.now(),
                "city": city,
                "temperature": weather_data['main']['temp'],
                "humidity": weather_data['main']['humidity'],
                "pressure": weather_data['main']['pressure'],
                "weather": weather_data['weather'][0]['description'],
                "wind_speed": weather_data['wind']['speed'],
                "season": get_current_season()
            }])
            weather_df = pd.concat([weather_df, new_data], ignore_index=True)

    # Save the DataFrame to CSV
    weather_df.to_csv('weather_data.csv', index=False)
    print("Updated weather DataFrame:")
    print(weather_df)

# For testing purposes, run the job function
job()
print("Final DataFrame:")
print(weather_df)

C:\Users\pawan\AppData\Local\Temp\ipykernel_14540\2422838524.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_df = pd.concat([weather_df, new_data], ignore_index=True)


Updated weather DataFrame:
                   timestamp       city  temperature humidity pressure  \
0 2024-10-02 16:53:07.146900      Delhi        35.05       41     1001   
1 2024-10-02 16:53:07.481456     Mumbai        30.99       74     1007   
2 2024-10-02 16:53:07.813644  Bangalore        29.99       55     1007   

         weather  wind_speed        season  
0           haze        4.12  Post-monsoon  
1           haze        4.63  Post-monsoon  
2  broken clouds        2.22  Post-monsoon  
Final DataFrame:
                   timestamp       city  temperature humidity pressure  \
0 2024-10-02 16:53:07.146900      Delhi        35.05       41     1001   
1 2024-10-02 16:53:07.481456     Mumbai        30.99       74     1007   
2 2024-10-02 16:53:07.813644  Bangalore        29.99       55     1007   

         weather  wind_speed        season  
0           haze        4.12  Post-monsoon  
1           haze        4.63  Post-monsoon  
2  broken clouds        2.22  Post-monsoon  


In [7]:
# Function to schedule the job
schedule.every().day.at("08:00").do(job)  # Set to run daily at 08:00 AM

Every 1 day at 08:00:00 do job() (last run: [never], next run: 2024-10-03 08:00:00)

In [ ]:
# Run the scheduled jobs
while True:
    schedule.run_pending()
    time.sleep(1)  # Wait for one second before checking again